In [1]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import os
import pandas as pd
from torchvision.models import densenet121

In [2]:
directory = 'C:/Users/Akv3/Downloads/archive_violence_detection/SCVD/SCVD_converted_sec_split/Train'

# TESTING VIDEO CAN BE READ
# Create a VideoCapture object
cap = cv2.VideoCapture(directory + '/' + 'Violence' + '/' + 'Violence001.avi')

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        # Display the resulting frame
        cv2.imshow('Frame', frame)
        # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    # Break the loop
    else: 
        break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

In [3]:
def calculate_optical_flow(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (224, 224))
    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[..., 1] = 255

    while (1):
        ret, frame2 = cap.read()
        if not ret:
            break
        frame2 = cv2.resize(frame2, (224, 224))
        next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(
            prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang*180/np.pi/2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        prvs = next
    cap.release()
    return rgb

In [4]:
# Normal = []
# Violence = []
# Weaponized = []
# for subdir in os.listdir(directory):
#     for filename in os.listdir(os.path.join(directory, subdir)):
#         if filename.endswith(".avi"):  # videos are in .avi format
#             video_path = os.path.join(directory, subdir, filename)
#             optical_flow = calculate_optical_flow(video_path)
#             if subdir == 'Normal':
#                 Normal.append(optical_flow)
#             elif subdir == 'Violence':
#                 Violence.append(optical_flow)
#             elif subdir == 'Weaponized':
#                 Weaponized.append(optical_flow)
# print(len(Normal), len(Violence), len(Weaponized))

In [5]:
model = densenet121(pretrained=True)
# DenseBlock 1
new_layers = nn.Sequential()
for i in range(len(model.features.denseblock1)):
    new_layer = nn.Sequential(
    nn.BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=(1, 1, 1), bias=False),
    nn.BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(128, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False))
    new_layers.add_module(str(i), new_layer)
model.features.denseblock1 = new_layers
# print(model.features.denseblock1)
print(len(model.features.denseblock1))
# DenseBlock 2
new_layers = nn.Sequential()
for i in range(len(model.features.denseblock2)):
    new_layer = nn.Sequential(
    nn.BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(128, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=(1, 1, 1), bias=False),
    nn.BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False))
    new_layers.add_module(str(i), new_layer)
model.features.denseblock2 = new_layers
# print(model.features.denseblock2)
print(len(model.features.denseblock2))
# DenseBlock 3
new_layers = nn.Sequential()
for i in range(len(model.features.denseblock3)):
    new_layer = nn.Sequential(
    nn.BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(256, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=(1, 1, 1), bias=False),
    nn.BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False))
    new_layers.add_module(str(i), new_layer)
model.features.denseblock3 = new_layers
# print(model.features.denseblock3)
print(len(model.features.denseblock3))
# DenseBlock 4
new_layers = nn.Sequential()
for i in range(len(model.features.denseblock4)):
    new_layer = nn.Sequential(
    nn.BatchNorm3d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(512, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=(1, 1, 1), bias=False),
    nn.BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(128, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False))
    new_layers.add_module(str(i), new_layer)
model.features.denseblock4 = new_layers
# print(model.features.denseblock4)
print(len(model.features.denseblock4))
# Transition Layers
new_layers = nn.Sequential(
    nn.BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=(1, 1, 1), bias=False),
    nn.AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0))
model.features.transition1 = new_layers
# print(model.features.transition1)
new_layers = nn.Sequential(
    nn.BatchNorm3d(128, eps=1e-05, momentum=0.1,
                   affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(128, 256, kernel_size=(1, 1, 1), stride=(
        1, 1, 1), padding=(1, 1, 1), bias=False),
    nn.AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0))
model.features.transition2 = new_layers
# print(model.features.transition2)
new_layers = nn.Sequential(
    nn.BatchNorm3d(256, eps=1e-05, momentum=0.1,
                   affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    nn.Conv3d(256, 512, kernel_size=(1, 1, 1), stride=(
        1, 1, 1), padding=(1, 1, 1), bias=False),
    nn.AvgPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0))
model.features.transition3 = new_layers
# print(model.features.transition3)
# Replace the first four layers
model.features.conv0 = nn.Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
model.features.norm0 = nn.BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
model.features.relu0 = nn.ReLU(inplace=True)
model.features.pool0 = nn.MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
model.features.norm5 = nn.BatchNorm3d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
print(model.features)
# Remove Classification Layer
model.classifier = nn.Identity()

c:\Users\Akv3\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Akv3\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


6
12
24
16
Sequential(
  (conv0): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): Sequential(
    (0): Sequential(
      (0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
      (2): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): ReLU(inplace=True)
      (5): Conv3d(128, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (1): Sequential(
      (0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
      (2): Conv3d(64, 128, kernel_

In [6]:
# Test model on one optical flow example
test_vid_path = directory + '/' + 'Violence' + '/' + 'Violence001.avi'
test_optical_flow = calculate_optical_flow(test_vid_path)
test_optical_flow = np.transpose(test_optical_flow, (2, 0, 1))
test_optical_flow = torch.tensor(test_optical_flow).float().unsqueeze(0)
# Repeat the single-channel image 3 times to create a 3-channel image
test_optical_flow = test_optical_flow.repeat(1, 3, 1, 1, 1)

print(test_optical_flow.shape)
output = model(test_optical_flow)
print(output.shape)

torch.Size([1, 3, 3, 224, 224])
torch.Size([1, 33280])
